In [102]:
import pandas as pd
import os

In [103]:
importFile = 'CombinedTestingFrame_A2_A3_O3.txt'
outName = 'CombinedTestingFrame_A2_A3_O3_short.txt'
df_import = pd.read_csv(importFile,names=['ID','label','extra'],sep='\t')

In [104]:
df_import['label'].value_counts()

label
14    36750
8     12950
0     10925
11     7850
3      7525
2      7075
9      5200
6      3875
7      2000
13     1700
12     1575
5       700
Name: count, dtype: int64

In [105]:
def extractNiftiFilepathAndSlicenum(df):
    ID = df['ID']
    split = ID.rsplit('__s',1)
    NiftiPath = split[0]
    NiftiPath = NiftiPath.replace('NIFTI_SLICES','NIFTI')+'.nii.gz'
    slicenum = int(split[1].split('.nii.gz')[0])
    return pd.Series({'NiftiPath':NiftiPath, 'slicenum':slicenum})

In [106]:
df_NS = df_import.apply(extractNiftiFilepathAndSlicenum,axis=1, result_type='expand')

In [107]:
df = df_import.merge(df_NS,left_index=True,right_index=True,how='inner')

In [108]:
vc = df['label'].value_counts()
vc

label
14    36750
8     12950
0     10925
11     7850
3      7525
2      7075
9      5200
6      3875
7      2000
13     1700
12     1575
5       700
Name: count, dtype: int64

In [109]:
min = vc.iloc[-1]
min

700

In [110]:
from sklearn.model_selection import  train_test_split

In [111]:
FinalFrame = pd.DataFrame(columns=['ID','label','extra'])
for l in df['label'].unique():
    temp = df[df['label']==l].groupby('NiftiPath')
    print(len(temp))
    try:
        keep, discard = train_test_split(list(range(len(temp))), train_size=int(min/25))
    except:
        keep = list(range(int(min/25)))
    for i,frame in enumerate(temp):
        if i in keep:
           FinalFrame =  pd.concat((FinalFrame,frame[1]),axis=0)

301
1470
518
68
437
155
28
283
314
208
80
63


In [112]:
FinalFrame['label'].value_counts()

label
3     700
14    700
8     700
13    700
0     700
6     700
5     700
2     700
11    700
9     700
7     700
12    700
Name: count, dtype: int64

In [113]:
FinalFrame = FinalFrame.reset_index(drop=True)

In [114]:
FinalFrame[['ID','label','extra']].to_csv(outName,header=False,index=False,sep='\t')

In [115]:
FinalFrame

,ID,label,extra,NiftiPath,slicenum
0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,4.0
1,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,24.0
2,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,2.0
3,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,8.0
4,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,22.0
...,...,...,...,...,...
8395,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS314...,12,0,data/OASIS/OASIS3/test/NIFTI/sub-OAS31460_sess...,8.0
8396,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS314...,12,0,data/OASIS/OASIS3/test/NIFTI/sub-OAS31460_sess...,22.0
8397,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS314...,12,0,data/OASIS/OASIS3/test/NIFTI/sub-OAS31460_sess...,23.0
8398,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS314...,12,0,data/OASIS/OASIS3/test/NIFTI/sub-OAS31460_sess...,4.0
